In [13]:
from __future__ import absolute_import
from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from sklearn.cross_validation import train_test_split
import pandas as pd
import numpy as np
from glob import glob
from skimage.io import imread, imshow, imsave
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
trainlabel = pd.read_csv('../Data/train.csv')

In [2]:
batch_size = 32
nb_classes = 447
nb_epoch = 3
data_augmentation = False

In [3]:
# shape of the image (SHAPE x SHAPE)
shapex, shapey = 200, 300
# number of convolutional filters to use at each layer
nb_filters = [32, 64]
# level of pooling to perform at each layer (POOL x POOL)
nb_pool = [2, 2]
# level of convolution to perform at each layer (CONV x CONV)
nb_conv = [3, 3]
# the MNIST images are greyscale
image_dimensions = 3

In [25]:
dloc = '../../BigData/kaggle-right-whale/imgs/'
#features = []
#for image in trainlabel['Image']:
#    dfile = dloc + image
#    splitfile = os.path.splitext(file)
#    fsmall = splitfile[0] + '_small' + splitfile[1]
#    imdata = imread(fsmall)
#    features.append(imdata.flatten())
features = []
foundlist = []
files = glob(dloc + '*_small.jpg')
for i in range(len(files)):
    si = str(i)
    tailloc = 'w_' + si + '_small.jpg'
    dfile = dloc + tailloc
    if 'w_' + si + '.jpg' in trainlabel['Image'].values and dfile in files:
        imdata = imread(dfile)
        features.append(np.reshape(imdata, (3, 200, 300)))
        trainlabelindex = trainlabel['Image'].values == 'w_' + si + '.jpg'
        trainlabelvalue = trainlabel['whaleID'].values[trainlabelindex][0]
        foundlist.append(trainlabelvalue)

In [26]:
features = np.array(features)

In [27]:
features.shape

(4543, 3, 200, 300)

In [28]:
vectorizer = CountVectorizer(min_df=1)

In [29]:
X = features
y = vectorizer.fit_transform(foundlist).toarray()

In [30]:
y.shape

(4543, 447)

In [31]:
# the data, shuffled and split between tran and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (3180, 3, 200, 300)
3180 train samples
1363 test samples


In [34]:
y.shape

(4543, 447)

In [35]:
model = Sequential()

In [36]:
model.add(Convolution2D(nb_filters[0], image_dimensions, nb_conv[0], nb_conv[0], border_mode='full'))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters[0], nb_filters[0], nb_conv[0], nb_conv[0]))
model.add(Activation('relu'))
model.add(MaxPooling2D(poolsize=(nb_pool[0], nb_pool[0])))
model.add(Dropout(0.25))

In [37]:
model.add(Convolution2D(nb_filters[1], nb_filters[0], nb_conv[0], nb_conv[0], border_mode='full'))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters[1], nb_filters[1], nb_conv[1], nb_conv[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(poolsize=(nb_pool[1], nb_pool[1])))
model.add(Dropout(0.25))

In [38]:
model.add(Flatten())

In [39]:
# the image dimensions are the original dimensions divided by any pooling
# each pixel has a number of filters, determined by the last Convolution2D layer
model.add(Dense(nb_filters[-1] * (shapex / nb_pool[0] / nb_pool[1]) * (shapey / nb_pool[0] / nb_pool[1]), 512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

In [40]:
model.add(Dense(512, nb_classes))
model.add(Activation('softmax'))

In [41]:
# let's train the model using SGD + momentum (how original).
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

In [ ]:
if not data_augmentation:
    print("Not using data augmentation or normalization")

    X_train = X_train.astype("float32")
    X_test = X_test.astype("float32")
    X_train /= 255
    X_test /= 255
    model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch, show_accuracy=True)
    score = model.evaluate(X_test, y_test, batch_size=batch_size)
    print('Test score:', score)

else:
    print("Using real time data augmentation")

    # this will do preprocessing and realtime data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=True,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=True,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)
    datagen.fit(X_train)

    for e in range(nb_epoch):
        print('-'*40)
        print('Epoch', e)
        print('-'*40)
        print("Training...")
        # batch train with realtime data augmentation
        progbar = generic_utils.Progbar(X_train.shape[0])
        for X_batch, Y_batch in datagen.flow(X_train, Y_train):
            loss = model.train_on_batch(X_batch, Y_batch)
            progbar.add(X_batch.shape[0], values=[("train loss", loss)])

        print("Testing...")
        # test time!
        progbar = generic_utils.Progbar(X_test.shape[0])
        for X_batch, Y_batch in datagen.flow(X_test, Y_test):
            score = model.test_on_batch(X_batch, Y_batch)
            progbar.add(X_batch.shape[0], values=[("test loss", score)])

Not using data augmentation or normalization
Epoch 0

In [22]:
test = pd.read_csv('../Data/test.csv')

In [23]:
test_image = makeImage(test.values)

In [24]:
ypredict = model.predict(test_image, batch_size=batch_size)

In [28]:
dfpredict = pd.DataFrame(np.argmax(ypredict, axis=1))
dfpredict.columns = ['Label']
dfpredict['ImageId'] = np.arange(28000) + 1
dfpredict.to_csv('../Data/predict_CNNbenchmark.csv', index=False)

In [27]:
np.argmax(ypredict, axis=1)

array([2, 0, 9, ..., 3, 9, 2])